In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import matplotlib.pyplot as plt
from tabulate import tabulate

# Membaca data dari file yang diunggah
data = pd.read_csv('Reddit_Data.csv')

# Menggantikan np.nan dengan string kosong ''
data['clean_comment'].fillna('', inplace=True)

# Mengambil hanya kolom 'clean_comment'
comments = data['clean_comment'].values

In [ ]:
# Menggunakan TF-IDF untuk mengubah teks menjadi representasi numerik
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(comments).toarray()

In [ ]:
# Membagi data menjadi data latih dan data uji
X_train, X_test, comments_train, comments_test = train_test_split(X, comments, test_size=0.2, random_state=42)

In [ ]:
# Membuat model Autoencoder
input_dim = X_train.shape[1]
encoding_dim = 32

input_layer = Input(shape=(input_dim,))
encoder = Dense(encoding_dim, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="sigmoid")(encoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer='adam', loss='mse')

# Melatih model Autoencoder
history = autoencoder.fit(X_train, X_train, epochs=20, batch_size=256, validation_split=0.2, verbose=1)

Epoch 1/20
94/94 [==============================] - 4s 36ms/step - loss: 0.1838 - val_loss: 0.0621
Epoch 2/20
94/94 [==============================] - 3s 30ms/step - loss: 0.0252 - val_loss: 0.0102
Epoch 3/20
94/94 [==============================] - 3s 29ms/step - loss: 0.0067 - val_loss: 0.0045
Epoch 4/20
94/94 [==============================] - 4s 46ms/step - loss: 0.0034 - val_loss: 0.0027
Epoch 5/20
94/94 [==============================] - 3s 29ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 6/20
94/94 [==============================] - 3s 30ms/step - loss: 0.0015 - val_loss: 0.0014
Epoch 7/20
94/94 [==============================] - 3s 29ms/step - loss: 0.0012 - val_loss: 0.0011
Epoch 8/20
94/94 [==============================] - 4s 38ms/step - loss: 9.4114e-04 - val_loss: 9.0324e-04
Epoch 9/20
94/94 [==============================] - 3s 36ms/step - loss: 7.8372e-04 - val_loss: 7.6561e-04
Epoch 10/20
94/94 [==============================] - 3s 30ms/step - loss: 6.7116e-04 - val_lo

In [ ]:
# Menggunakan model untuk mendeteksi anomali
X_train_pred = autoencoder.predict(X_train)
mse = np.mean(np.power(X_train - X_train_pred, 2), axis=1)
threshold = np.percentile(mse, 95)  # Menentukan threshold dari data latih

X_test_pred = autoencoder.predict(X_test)
mse_test = np.mean(np.power(X_test - X_test_pred, 2), axis=1)

# Menandai anomali berdasarkan threshold
anomalies = mse_test > threshold

233/233 [==============================] - 1s 3ms/step


In [ ]:
# Tampilkan jumlah anomali terdeteksi dengan format yang menonjol
print(f'\n===============================================')
print(f' Jumlah anomali terdeteksi: {np.sum(anomalies)} ')
print(f'===============================================\n')

# Menampilkan komentar anomali dalam tabel menggunakan tabulate
anomalous_comments = comments_test[anomalies]
anomalous_comments_table = pd.DataFrame(anomalous_comments[:10], columns=['Anomalous Comments'])

# Menggunakan tabulate untuk tampilan tabel yang lebih baik
table = tabulate(anomalous_comments_table, headers='keys', tablefmt='pretty', showindex=False)
print(table)


 Jumlah anomali terdeteksi: 452 

+-------------------------------------------------------------------+
|                        Anomalous Comments                         |
+-------------------------------------------------------------------+
|                                yay                                |
|                               hell                                |
|              bas inka yahi randi rona chalta rahega               |
|                            disgraceful                            |
|                         bahot hua samman                          |
|                  sandking notice the crab claws                   |
|                              shikari                              |
|                              dumbass                              |
|                             scum bag                              |
| 孤单寂寞觉得冷 难道要刻意假装说“印度威胁到中国制造业”才叫清醒么？ |
+-------------------------------------------------------------------+
